In [1]:
from gurobipy import *
from prettytable import PrettyTable 
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')

Demand = 700
GenTotal = quicksum(P[i] for i in Gen)
GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
EC.setObjective(GenCost, GRB.MINIMIZE)
DemandConst = EC.addConstr( GenTotal == Demand, name = 'DemandConst')

#Algorithm used to solve continuous models or the root node of a MIP model. 
#Options are: -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 
#              3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex.
EC.Params.Method=-1
EC.setParam( 'OutputFlag', False )
EC.optimize()

print('Total Cost:', "€",round(EC.objVal,2))
print('Cost of Next MW:', "€",round(DemandConst.getAttr('PI'),2))

Results = PrettyTable(['Generator','Lower (MW)','Value (MW)','Upper (MW)', 'Marginal (€)'])
for v in EC.getVars():
    Results.add_row([v.varName, v.LB,round((v.x),3), v.UB,round(v.getAttr('RC'),3)])
print(Results)

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1
Total Cost: € 413063.58
Cost of Next MW: € 1218.96
+-------------+------------+------------+------------+--------------+
|  Generator  | Lower (MW) | Value (MW) | Upper (MW) | Marginal (€) |
+-------------+------------+------------+------------+--------------+
| Power[Gen1] |    28.0    |  199.827   |   206.0    |     -0.0     |
| Power[Gen2] |    90.0    |  148.258   |   284.0    |     0.0      |
| Power[Gen3] |    68.0    |  148.569   |   189.0    |     -0.0     |
| Power[Gen4] |    76.0    |  150.345   |   266.0    |     0.0      |
| Power[Gen5] |    19.0    |    53.0    |    53.0    |   -781.503   |
+-------------+------------+------------+------------+--------------+


In [2]:
method =1
EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }
P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')

Demand = 700
GenTotal = quicksum(P[i] for i in Gen)
print(GenTotal)

<gurobi.LinExpr: Power[Gen1] + Power[Gen2] + Power[Gen3] + Power[Gen4] + Power[Gen5]>


In [51]:
from gurobipy import *
from prettytable import PrettyTable
from numpy import array, concatenate,append

EC = Model('Economic Dispatch')
Gen = ['Gen1','Gen2', 'Gen3', 'Gen4', 'Gen5']
GenAt = {Gen[0]:{'a':3 ,'b':20 ,'c':100 , 'Pmin':28 ,'Pmax':206 },
      Gen[1]:{'a':4.05 ,'b':18.07 ,'c':98.87 , 'Pmin':90 ,'Pmax':284 },
      Gen[2]:{'a':4.05 ,'b':15.55 ,'c':104.26 , 'Pmin':68 ,'Pmax':189 },
      Gen[3]:{'a':3.99 ,'b':19.21 ,'c':107.21 , 'Pmin':76 ,'Pmax':266 },  
      Gen[4]:{'a':3.88 ,'b':26.18 ,'c':95.31 , 'Pmin':19 ,'Pmax':53 }  }



Iter =11
Load=[]
TotalLoad=[]
Objective=[]
Var=[]
for c in range(Iter):
    Demand = quicksum(GenAt[i]['Pmin'] for i in Gen) + \
    ((c)/(Iter-1))*quicksum(GenAt[i]['Pmax'] - GenAt[i]['Pmin'] for i in Gen)
    Load.append(Demand)




for k in range(Iter):
   
    P = EC.addVars(Gen,lb = [GenAt[i]['Pmin'] for i in Gen], ub = [GenAt[i]['Pmax'] for i in Gen],name = 'Power')
    GenTotal = quicksum(P[i] for i in Gen)
    GenCost = quicksum(GenAt[i]['c']+GenAt[i]['b']*P[i]+GenAt[i]['a']*P[i]*P[i] for i in Gen)
    EC.setObjective(GenCost, GRB.MINIMIZE)
    target = EC.addConstr( GenTotal == Load[k], name = 'DemandConst')
   
    

    EC.Params.Method=-1
    EC.setParam( 'OutputFlag', False )
    EC.optimize()
    TotalLoad.append(Load[k].getValue())
    v = EC.getVars()
    Obj = EC.objVal
    Objective.append(EC.objVal)
    

    status = EC.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE \
    or status == GRB.Status.UNBOUNDED:
        EC.computeIIS()
        EC.write("model.ilp")

Parameter Method unchanged
   Value: -1  Min: -1  Max: 5  Default: -1


In [50]:
from numpy import concatenate, around

for v in EC.getVars():
        Var.append(v.x)
Power = array(Var).reshape(Iter,len(Gen))

TotalLoad = array(TotalLoad,  ndmin=2)
TotalLoad = TotalLoad.T

Objective = array(Objective, ndmin =2)
Objective = Objective.T

Values = concatenate([Power, TotalLoad, Objective], axis = 1)
Values =  around(Values, decimals = 2)


Results = PrettyTable(['Period','Gen 1','Gen 2','Gen 3', 'Gen 4', 'Gen 5', 'Load', 'Objective Value'])
for v in range(0,len(Values)):
    Results.add_row([v, Values[v,0], Values[v,1], Values[v,2], Values[v,3],Values[v,4],Values[v,5], Values[v,6]])
print(Results)


+--------+-------+-------+-------+-------+-------+-------+-----------------+
| Period | Gen 1 | Gen 2 | Gen 3 | Gen 4 | Gen 5 |  Load | Objective Value |
+--------+-------+-------+-------+-------+-------+-------+-----------------+
|   0    |  28.0 |  90.0 |  68.0 |  76.0 |  19.0 | 281.0 |     84037.85    |
|   1    |  99.7 |  90.0 |  68.0 |  76.0 |  19.0 | 352.7 |    105758.02    |
|   2    | 171.4 |  90.0 |  68.0 |  76.0 |  19.0 | 424.4 |    147715.94    |
|   3    |  28.0 |  90.0 |  93.1 | 266.0 |  19.0 | 496.1 |    203091.65    |
|   4    |  28.0 |  90.0 | 164.8 | 266.0 |  19.0 | 567.8 |    268132.17    |
|   5    |  28.0 | 284.0 |  68.0 | 240.5 |  19.0 | 639.5 |    342712.41    |
|   6    | 206.0 | 284.0 |  68.0 | 134.2 |  19.0 | 711.2 |    426832.36    |
|   7    | 206.0 | 284.0 |  68.0 | 205.9 |  19.0 | 782.9 |    522137.81    |
|   8    |  62.6 | 284.0 | 189.0 | 266.0 |  53.0 | 854.6 |    631691.67    |
|   9    | 134.3 | 284.0 | 189.0 | 266.0 |  53.0 | 926.3 |     760550.5    |